# Analyzing the Value Premium

## Introduction
In 1992, Eugene Fama and Kenneth French introduced a Three-Factor Model that extends William Sharpe's CAPM with a Size Factor and a Value Factor:
<br><br>
$$
E(r) = \alpha + \beta_{1} *  (MKT-R_f) + \beta_{2} * SMB + \beta_{3} * HML
$$
<br>
Given the parsimonous structure, the theoretical existence of state variables that investors want to hedge against and the poor performance of the CAPM, it has become, together with Fama/French's extended 5-Factor Model, the standard  model to price the cross-section of equity returns.
<br>
However, since the publication of the paper, both factors failed to deliver any premium out-of-sample and the value premium (HML) saw the largest drawdown in history since the financial crisis 2008/09, erasing a third of its 100-year cumulative outperformance. Sceptics claim that both premia are not related to risk and should unsurpringly vanish once their existence becomes common knowledge.
<br>
This notebook provides an analysis of the factors that drive the value premium and shows that it cannot be concluded that the value premium is dead.

**Data sources:**
* Kenneth French's Academic Portfolios (https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html)
  * In particular *Fama/French 5 Factors* and *6 Portfolios Formed on Size and Book-to-Market (2 x 3)*
* Robert Shiller's History of Bond Rates (http://www.econ.yale.edu/~shiller/data.htm)

## Content

#### Setting up the Data

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.api import OLS, add_constant
from statsmodels.tsa.arima.model import ARIMA
from finance_data import FrenchReader, ShillerReader

In [7]:
bond_rate = ShillerReader.cape()["10-Year Interest Rate"] / 100
ff5 = FrenchReader("F-F_Research_Data_5_Factors_2x3").read()["Main"] / 100
mom = FrenchReader("F-F_Momentum_Factor").read()["Main"] / 100
df = pd.concat([ff5, mom, bond_rate], axis=1).dropna()
df

,Mkt-RF,SMB,HML,RMW,CMA,RF,Mom,10-Year Interest Rate
1963-07-01,-0.0039,-0.0044,-0.0089,0.0068,-0.0123,0.0027,0.0097,0.0402
1963-08-01,0.0507,-0.0075,0.0168,0.0036,-0.0034,0.0025,0.0101,0.0400
1963-09-01,-0.0157,-0.0055,0.0008,-0.0071,0.0029,0.0027,0.0019,0.0408
1963-10-01,0.0253,-0.0137,-0.0014,0.0280,-0.0202,0.0029,0.0312,0.0411
1963-11-01,-0.0085,-0.0089,0.0181,-0.0051,0.0231,0.0027,-0.0074,0.0412
...,...,...,...,...,...,...,...,...
2021-08-01,0.0290,-0.0079,-0.0013,-0.0026,-0.0167,0.0000,0.0252,0.0128
2021-09-01,-0.0437,0.0125,0.0509,-0.0194,0.0208,0.0000,0.0157,0.0137
2021-10-01,0.0665,-0.0269,-0.0044,0.0174,-0.0148,0.0000,0.0321,0.0158
2021-11-01,-0.0155,-0.0174,-0.0053,0.0738,0.0160,0.0000,0.0082,0.0156


In [5]:
portfolio_data = FrenchReader("6_Portfolios_2x3").read()
for key in portfolio_data.keys():
    print(key)

Average Value Weighted Returns Monthly
Average Equal Weighted Returns Monthly
Average Value Weighted Returns Annual
Average Equal Weighted Returns Annual
Number of Firms in Portfolios
Average Market Cap
Value Weight Average of BE/ME
Value Weight Average of BE_FYt-1/ME_June
Value Weight Average of OP
Value Weight Average of investment
